<a href="https://colab.research.google.com/github/aroonalok/zoho-apis/blob/changes/ZohoBooksReports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime
from google.colab import files
import IPython
import openpyxl
from openpyxl.styles import Font, Color, Border, Side
import pandas as pd
import pytz
import re
import requests

# Configuration

*   Go to https://api-console.zoho.in/
*   Click on the `Self Client`
*   Copy `Client ID` and `Client Secret` from `Client Secret` tab.
*   Organization ID can be left unconfigured
*   Auth Token can be left unconfigured if you have the downloaded Token file (`authorization_token.txt`) in your local machine.

In [ ]:
# @title Access Config

ORGANIZATION_ID = 0 # @param {type:"integer"}
AUTH_TOKEN = "" # @param {type:"string"}
CLIENT_ID = "" # @param {type:"string"}
CLIENT_SECRET = "" # @param {type:"string"}

if not (CLIENT_ID and CLIENT_SECRET):
  raise AssertionError("Fill up Client ID and Client Secret values!")

In [ ]:
# @title Date Config

FROM_DATE = '' # @param {type:"date"}
TO_DATE = '' # @param {type:"date"}

DATE_FORMAT = '%Y-%m-%d'
if datetime.strptime(FROM_DATE, DATE_FORMAT) > datetime.strptime(TO_DATE, DATE_FORMAT):
  raise AssertionError("TO_DATE should be greater than FROM_DATE ")

In [ ]:
# @title API Config

MAX_RECORDS_PER_PAGE = 1000
IST_TIMEZONE = pytz.timezone('Asia/Kolkata')

API_CALLS_COUNTER = 0

ROOT_API_ENDPOINT = "https://www.zohoapis.in/books/v3/"
OAUTH_ROOT_URL = "https://accounts.zoho.in/oauth/v2/token?"
OAUTH_REDIRECT_URI = "http://www.zoho.in/books"

In [ ]:
# @title Report Config

DOWNLOAD_EXCEL_REPORT = False # @param {type:"boolean"}
DOWNLOAD_HTML_REPORT = False # @param {type:"boolean"}

GET_GST_REPORT = False # @param {type:"boolean"}

PAYMENT_MODE_CREDIT = "Credit"
FROM_TO_DATE = "FROM: {from_date} TO: {to_date}"
FOR_DATE = "FOR: {for_date}"

DAILY_SALES_REPORT_ID = "Sales"
SALES_TYPE_REPORT_ID = "Sales Type"
INTRA_STATE_TAX_REPORT_ID = "GST"
INTRA_STATE_TAX_SUMMARY_REPORT_ID = "GST Summary"

NO_DATA_MESSAGE = "NO DATA PRESENT {time_range}"

ITEM_TYPE_BOOK = "Book"
ITEM_TYPE_ARTICLE = "Article"

REPORT_TAB_TITLE = "" # @param {type:"string"}
LOGO_URL = "" # @param {type:"string"}
ORGANIZATION_NAME = "" # @param {type:"string"}
ADDRESS_LINE1 =  "" # @param {type:"string"}
ADDRESS_LINE2 = '' # @param {type:"string"}
PHONE = '' # @param {type:"string"}
EMAIL = '' # @param {type:"string"}
REPORT_TITLE = ""

In [ ]:
# @title HTML Templates

TABLE_CAPTION_TEMPLATE = '<caption>{caption}</caption>'
TABLE_TEMPLATE = '<table>{table_caption}{table_head}{table_body}</table>'
HTML_DOC_TEMPLATE = '''
          <!DOCTYPE html>
          <html lang="en">
          <head>
              <meta charset="UTF-8">
              <meta name="viewport" content="width=device-width, initial-scale=1.0">
              <title>{report_tab_title}</title>
              <style>
                  body {{
                      font-family: Arial, sans-serif;
                      margin: 0;
                      padding: 0;
                  }}

                  .header {{
                      padding: 20px;
                      text-align: center;
                  }}

                  .logo {{
                      display: inline-block;
                      vertical-align: middle;
                  }}

                  .company-info {{
                      display: inline-block;
                      vertical-align: middle;
                      margin-left: 20px;
                  }}

                  .company-name {{
                      font-size: 24px;
                      font-weight: bold;
                      margin-bottom: 5px;
                  }}

                  .company-address {{
                      font-size: 16px;
                  }}

                  .report-content {{
                      padding: 20px;
                      text-align: center;
                  }}

                  table {{
                      margin: auto;
                      border-collapse: collapse;
                      width: 80%;
                      margin-bottom: 20px;
                  }}

                  th, td {{
                      border: 1px solid black;
                      padding: 8px;
                  }}

                  caption {{
                      font-size: 19px;
                      font-weight: bold;
                      caption-side: top;
                      margin-bottom: 10px;
                  }}

                  h1 {{
                      margin-bottom: 30px;
                  }}
              </style>
          </head>
          <body>
              <div class="header">
                  <img src="{logo_url}" alt="Logo" class="logo" width="125" height="125">
                  <div class="company-info">
                      <div class="company-name">{org_name}</div>
                      <div class="company-address">
                          <div>{address_line1}</div>
                          <div>{address_line2}</div>
                          <div>{phone}</div>
                          <div>{email}g</div>
                      </div>
                  </div>
              </div>
              <div class="report-content">
                  <h1>{report_title}</h1>
                  {tables}
              </div>
          </body>
          </html>
        '''

# Code

In [ ]:
# @title Utils

def formatTimestamp(timestamp):
  timestamp_datetime = datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%S%z')
  ist_datetime = timestamp_datetime.astimezone(IST_TIMEZONE)
  return ist_datetime.strftime("%d-%m-%Y %H:%M")

def calculate_tax_inclusive_amount(item):
  discount = float(item['discount'].strip('%'))/100 if item['discount'] else 0.0
  return item['rate']*item['quantity']*(1-discount)

def getItemType(item):
  return ITEM_TYPE_BOOK if item['hsn_or_sac'] == '49011010' else ITEM_TYPE_ARTICLE

def formatDateForReport(date):
  return datetime.strptime(date, DATE_FORMAT).strftime('%d-%m-%Y')

def getGstValues(item):
  cgst_amount, cgst_percent, sgst_amount, sgst_percent = 0.0, '0%', 0.0, '0%'
  for tax in item['line_item_taxes']:
    if tax['tax_name'].startswith("CGST"):
      cgst_amount, cgst_percent = tax['tax_amount'], re.search('\d*%', tax['tax_name']).group()
    elif tax['tax_name'].startswith("SGST"):
      sgst_amount, sgst_percent = tax['tax_amount'], re.search('\d*%', tax['tax_name']).group()
  return cgst_amount, cgst_percent, sgst_amount, sgst_percent

In [ ]:
# @title OAuth

def loadOauthToken():
  global AUTH_TOKEN
  print("Select Authorization Token file:")
  uploaded_auth_token = files.upload()
  AUTH_TOKEN = next(iter(uploaded_auth_token.values())).decode()

def renewOauthToken(refresh_token, client_id, client_secret):
  global API_CALLS_COUNTER
  params = []
  params.append("refresh_token={REFRESH_TOKEN}".format(REFRESH_TOKEN=refresh_token))
  params.append("client_id={CLIENT_ID}".format(CLIENT_ID=client_id))
  params.append("client_secret={CLIENT_SECRET}".format(CLIENT_SECRET=client_secret))
  params.append("redirect_uri={REDIRECT_URI}".format(REDIRECT_URI=OAUTH_REDIRECT_URI))
  params.append("grant_type=refresh_token")
  url = OAUTH_ROOT_URL+"&".join(params)
  payload = {}
  headers = {}
  response = requests.request("POST", url, headers=headers, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()

def createSession(access_token):
  session = requests.Session()
  session.headers.update({'Authorization': 'Zoho-oauthtoken {access_token}'.format(access_token=access_token)})
  return session

In [ ]:
# @title Organization

def getOrganizationId(session):
  '''
    Only takes the first organization from the list of orgs.
  '''
  global API_CALLS_COUNTER, ORGANIZATION_ID, ORGANIZATION_NAME
  url = ROOT_API_ENDPOINT+'organizations'
  payload = {}
  response = session.get(url,data=payload)
  API_CALLS_COUNTER += 1
  org_info = response.json()
  org = org_info['organizations'][0]
  print("Using Organization: {name}".format(name=org['name']))
  ORGANIZATION_ID = org['organization_id']
  if not ORGANIZATION_NAME:
    ORGANIZATION_NAME = org['name']

In [ ]:
# @title Invoices

def getInvoices(session, from_date=None, to_date=None, additional_params=[]):
  global API_CALLS_COUNTER
  params = additional_params
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  params.append("per_page={MAX_RECORDS_PER_PAGE}".format(MAX_RECORDS_PER_PAGE=MAX_RECORDS_PER_PAGE))
  params.append("sort_column=created_time")
  if from_date is not None:
    params.append("date_start={FROM_DATE}".format(FROM_DATE=from_date))
  if to_date is not None:
    params.append("date_end={TO_DATE}".format(TO_DATE=to_date))
  url = ROOT_API_ENDPOINT+"invoices?"+"&".join(params)

  results = []
  payload = {}
  send_request = True
  current_page = 1
  while send_request:
    response = session.get(url+"&page={PAGE}".format(PAGE=current_page),data=payload)
    API_CALLS_COUNTER += 1
    #perform exception handling here
    results += response.json()['invoices']
    send_request = response.json()['page_context']['has_more_page']
    current_page += 1

  return pd.DataFrame.from_records(results) if results else None

def getInvoiceDetails(session, invoice_id):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  url = ROOT_API_ENDPOINT+"invoices/{invoice_id}?".format(invoice_id=invoice_id)+"&".join(params)
  payload = {}
  response = session.get(url, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()
  #perform exception handling here

def getPaymentsForInvoice(session, invoice_id):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  url = ROOT_API_ENDPOINT+"invoices/{invoice_id}/payments?".format(invoice_id=invoice_id)+"&".join(params)
  payload = {}
  response = session.get(url, data=payload)
  API_CALLS_COUNTER += 1
  return response.json()
  #perform exception handling here

In [ ]:
# @title Payments

def getPayments(session, from_date=None, to_date=None):
  global API_CALLS_COUNTER
  params = []
  params.append("organization_id={ORGANIZATION_ID}".format(ORGANIZATION_ID=ORGANIZATION_ID))
  params.append("per_page={MAX_RECORDS_PER_PAGE}".format(MAX_RECORDS_PER_PAGE=MAX_RECORDS_PER_PAGE))
  params.append("sort_column=created_time")
  if from_date is not None:
    params.append("date_start={FROM_DATE}".format(FROM_DATE=from_date))
  if to_date is not None:
    params.append("date_end={TO_DATE}".format(TO_DATE=to_date))
  url = ROOT_API_ENDPOINT+"customerpayments?"+"&".join(params)

  results = []
  payload = {}
  send_request = True
  current_page = 1
  while send_request:
    response = session.get(url+"&page={PAGE}".format(PAGE=current_page), data=payload)
    API_CALLS_COUNTER += 1
    #perform exception handling here
    results += response.json()['customerpayments']
    send_request = response.json()['page_context']['has_more_page']
    current_page += 1

  return pd.DataFrame.from_records(results) if results else None

In [ ]:
# @title Class Definitions

class Table(object):
  '''
    Represents a data table displayed in a report.
  '''
  def __init__(self, id, caption, from_date, to_date, show_index):
    self.id = id
    self.caption = caption
    self.from_date = from_date
    self.to_date = to_date
    self.show_index = show_index
    self.data_rows = []
    self.df = None

  def append_row(self):
    # to be implemented in derived classes.
    pass

  def prepare_df(self):
    self.df = pd.DataFrame(data = self.data_rows)
    self.data_rows = []

  def has_data(self):
    return not (self.df is None or self.df.empty)

  def format_caption(self, time_range):
    self.caption = self.caption.format(time_range=time_range)

  def get_html(self):
    output = ''
    if not self.has_data():
      return output
    html_string = self.df.to_html(index=self.show_index, justify='center')
    soup = BeautifulSoup(html_string, 'html.parser')
    table = soup.find('table')
    if not table:
      return output
    thead = soup.find('thead')
    tbody = soup.find('tbody')
    if thead and tbody:
      output = TABLE_TEMPLATE.format(table_caption=TABLE_CAPTION_TEMPLATE.format(caption=self.caption),
                                     table_head=thead,table_body=tbody)
    return output

  def get_excel(self):
    if not self.has_data():
      return None
    table = self.df.copy(deep=True)
    table.columns = pd.MultiIndex.from_product([[self.caption], self.df.columns])
    return table

class DailySalesTable(Table):
  def __init__(self, from_date, to_date):
    super(DailySalesTable, self).__init__(id=DAILY_SALES_REPORT_ID,
                     caption="DAILY SALES REPORT {time_range}",
                     from_date=from_date, to_date=to_date, show_index=False)

  def append_row(self,**kwargs):
    invoice = kwargs.get('invoice', None)
    payment_mode = kwargs.get('payment_mode', None)
    if invoice and payment_mode:
      self.data_rows.append({'Invoice No.' : invoice['invoice_number'],
                            'Time' : formatTimestamp(invoice['created_time']),
                            'Payment Mode' : payment_mode,
                            'Amount (including GST)' : round(invoice['bcy_total'])})


class SalesTypeTable(Table):
  def __init__(self, from_date, to_date):
    super(SalesTypeTable, self).__init__(id=SALES_TYPE_REPORT_ID,
                     caption="SALES TYPE SUMMARY REPORT {time_range}",
                     from_date=from_date, to_date=to_date, show_index=True)

  def append_row(self,**kwargs):
    item = kwargs.get('item', None)
    payment_mode = kwargs.get('payment_mode', None)
    if item and payment_mode:
      self.data_rows.append({'Item Type' : getItemType(item),
                             'Amount (including GST)' : round(calculate_tax_inclusive_amount(item)),
                             'Payment Mode' : payment_mode})

  def prepare_df(self):
    super(SalesTypeTable, self).prepare_df()
    if self.has_data():
      self.df = self.df.groupby(['Item Type', 'Payment Mode']).sum()

class IntraStateTaxTable(Table):
  def __init__(self, from_date, to_date):
    super(IntraStateTaxTable, self).__init__(id=INTRA_STATE_TAX_REPORT_ID,
                     caption="INTRA STATE TAX REPORT {time_range}",
                     from_date=from_date, to_date=to_date, show_index=False)

  def append_row(self,**kwargs):
    item = kwargs.get('item', None)
    invoice = kwargs.get('invoice', None)
    cgst_amount, cgst_percent = kwargs.get('cgst', None)
    sgst_amount, sgst_percent = kwargs.get('sgst', None)

    if item and invoice:
      self.data_rows.append({'Date' : invoice['date'],
                            'Invoice No.' : invoice['invoice_number'],
                            'SKU' : item['sku'],
                            'Amount (excluding GST)' : round(item['item_total']),
                            'CGST Amount' : cgst_amount,
                            'SGST Amount' : sgst_amount,
                            'CGST Percent' : cgst_percent,
                            'SGST Percent' : sgst_percent,
                            'Amount (including GST)' : round(calculate_tax_inclusive_amount(item))})

class IntraStateTaxSummaryTable(Table):
  def __init__(self, intra_state_tax_table : IntraStateTaxTable):
    self.intra_state_tax_table = intra_state_tax_table
    super(IntraStateTaxSummaryTable, self).__init__(id=INTRA_STATE_TAX_SUMMARY_REPORT_ID,
                     caption="INTRA STATE TAX SUMMARY REPORT {time_range}",
                     from_date=intra_state_tax_table.from_date,
                     to_date=intra_state_tax_table.to_date,
                     show_index=True)
  def prepare_df(self):
    if self.intra_state_tax_table.has_data():
      self.df = self.intra_state_tax_table.df[['Amount (excluding GST)',
                                            'CGST Amount', 'SGST Amount',
                                            'Amount (including GST)']].sum().to_frame(name="Total")

class TotalAmountTable(Table):
  def __init__(self, daily_sales_table : DailySalesTable,
               sales_type_table : SalesTypeTable):
    self.daily_sales_table = daily_sales_table
    self.sales_type_table = sales_type_table
    super(TotalAmountTable, self).__init__(id=TOTAL_AMOUNT_REPORT_ID,
                     caption="TOTAL AMOUNTS REPORT {time_range}",
                     from_date=daily_sales_table.from_date,
                     to_date=daily_sales_table.to_date,
                     show_index=True)
  def prepare_df(self):
    totals = {}

    if self.daily_sales_table.has_data():
      totals["Amount (including GST)"] = self.daily_sales_table.df['Amount (including GST)'].sum()

    if self.sales_type_table.has_data():
      temp_df =  self.sales_type_table.df.copy(deep=True)
      temp_df.index = temp_df.index.droplevel('Item Type')
      temp_df = temp_df.groupby('Payment Mode').sum()
      for idx in temp_df.index:
        totals[idx] = temp_df['Amount (including GST)'][idx]

    self.df = pd.DataFrame(data=totals, index=["Total"]).T

class Report(object):
  def __init__(self, tables : list[Table], from_date, to_date):
    self.tables = tables
    self.from_date = from_date
    self.to_date = to_date
    self.filename = "report-from-{from_date}-to-{to_date}.{extension}"
    self.html_report = ''

  def prepare_html_report(self, report_tab_title, logo_url, org_name,
                          address_line1, address_line2, phone, email, time_range):
    tables_html = ''.join([table.get_html() for table in self.tables])
    if tables_html:
      self.html_report = HTML_DOC_TEMPLATE.format(report_tab_title=report_tab_title,
                                    logo_url=logo_url, org_name=org_name,
                                    address_line1=address_line1, address_line2=address_line2,
                                    phone=phone, email=email, report_title=REPORT_TITLE.format(time_range=time_range),
                                    tables=tables_html)
    else:
      self.html_report = HTML_DOC_TEMPLATE.format(report_tab_title=report_tab_title,
                                    logo_url=logo_url, org_name=org_name,
                                    address_line1=address_line1, address_line2=address_line2,
                                    phone=phone, email=email, report_title=NO_DATA_MESSAGE.format(time_range=time_range),
                                    tables=tables_html)

  def save_html(self, download, report_tab_title, logo_url, org_name, address_line1,
                address_line2, phone, email, time_range):
    self.prepare_html_report(report_tab_title, logo_url, org_name, address_line1,
                             address_line2, phone, email, time_range)
    html_filename = self.filename.format(from_date=self.from_date,
                                         to_date=self.to_date,
                                         extension='html')
    with open(html_filename, "w") as html_file:
      html_file.write(self.html_report)

    if download:
      files.download(html_filename)

  def display_html_report(self):
    display(IPython.display.HTML(self.html_report))


  def save_excel(self, download):
    if not self.tables:
      return
    excel_filename = self.filename.format(from_date=self.from_date,
                                          to_date=self.to_date,
                                          extension='xlsx')
    with pd.ExcelWriter(excel_filename) as writer:
      for table in self.tables:
        if not table.show_index:
          table.df.reset_index()
        table.get_excel().to_excel(writer, sheet_name=table.id)

    thin = Side(border_style="thin", color="000000")
    wb = openpyxl.load_workbook(excel_filename)
    for sheet in wb.sheetnames:
      ws = wb[sheet]
      for row in ws[ws.dimensions]:
        for cell in row:
          cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)
    wb.save(excel_filename)

    if download:
      files.download(excel_filename)

In [ ]:
# @title Report Creation

def fetchDataForReporting(session, from_date=None, to_date=None):
  data = []
  invoices = getInvoices(session, from_date, to_date)
  if invoices is None:
    return data

  invoice_table = DailySalesTable(from_date,to_date)
  sales_type_table = SalesTypeTable(from_date,to_date)
  intra_state_tax_table = IntraStateTaxTable(from_date, to_date)
  for invoice_id in invoices['invoice_id']:
    invoice = getInvoiceDetails(session, invoice_id)['invoice']
    payments = getPaymentsForInvoice(session, invoice_id)['payments']
    #assert len(payments) <= 1
    payment_mode = payments[0]['payment_mode'] if payments else PAYMENT_MODE_CREDIT
    invoice_table.append_row(invoice=invoice, payment_mode=payment_mode)
    for item in invoice['line_items']:
      sales_type_table.append_row(item=item, payment_mode=payment_mode)
      if not GET_GST_REPORT:
        continue
      cgst_amount, cgst_percent, sgst_amount, sgst_percent = getGstValues(item)
      intra_state_tax_table.append_row(item=item, invoice=invoice,
                                       cgst=(cgst_amount, cgst_percent),
                                       sgst=(sgst_amount, sgst_percent))

  intra_state_tax_summary_table = IntraStateTaxSummaryTable(intra_state_tax_table)
  total_amount_table = TotalAmountTable(invoice_table, sales_type_table)
  data = [invoice_table, sales_type_table, total_amount_table, intra_state_tax_table, intra_state_tax_summary_table]
  for table in data:
    table.prepare_df()
  data = [table for table in data if table.has_data()]
  return data

def generateReport(session, from_date=None, to_date=None):
  report_data = fetchDataForReporting(session, from_date, to_date)

  time_range = None
  if from_date == to_date:
    time_range = FOR_DATE.format(for_date=formatDateForReport(from_date))
  else:
    time_range = FROM_TO_DATE.format(from_date=formatDateForReport(from_date),
                                     to_date=formatDateForReport(to_date))

  for table in report_data:
    table.format_caption(time_range=time_range)

  report = Report(report_data, from_date, to_date)
  report.save_html(DOWNLOAD_HTML_REPORT,
                  report_tab_title=REPORT_TAB_TITLE,
                  logo_url=LOGO_URL,
                  org_name=ORGANIZATION_NAME,
                  address_line1=ADDRESS_LINE1,
                  address_line2=ADDRESS_LINE2,
                  phone=PHONE,
                  email=EMAIL,
                  time_range=time_range)
  report.save_excel(DOWNLOAD_EXCEL_REPORT)

  return report

# Daily Sales Summary Report Generation

**Policies**

 - For a given Invoice, there can be only one payment mode. For example, an invoice cannot have partial payments via Card(s) and Cash / UPI.
 - All Credit settlement is done fully and at once. No installments. As in the above case, there can not be partial credit and partial payment for an Invoice.

In [ ]:
# @title Initialization

if not AUTH_TOKEN:
  loadOauthToken()

oauth_token = renewOauthToken(AUTH_TOKEN,CLIENT_ID, CLIENT_SECRET)
session = createSession(oauth_token['access_token'])

if not ORGANIZATION_ID:
  getOrganizationId(session)

In [ ]:
# @title Report

report = generateReport(session=session, from_date=FROM_DATE, to_date=TO_DATE)
report.display_html_report()

# API Calls

In [ ]:
print("API calls made = {count}".format(count=API_CALLS_COUNTER))